In [99]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import json
from bson import json_util, ObjectId

from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool

load_dotenv()

client = MongoClient("mongodb://localhost:27017/")
db = client["ai_learning_platform"]
courses_collection = db["courses"]
users_collection = db["users"]

@tool
def search_courses(query: str) -> str:
    """
    Use this tool ONLY for questions about course details like lessons, content, instructors, or descriptions.
    The input must be a valid Python command string using methods like find or aggregate on the 'courses_collection' variable.
    """
    print(f"  Executing on 'courses' collection: {query}")
    try:
        local_vars = {"courses_collection": courses_collection, "ObjectId": ObjectId}
        result = eval(query, {"__builtins__": {}}, local_vars)
        if isinstance(result, int): return str(result)
        result_list = list(result)
        if not result_list: return "No courses found matching the query."
        return json.dumps(result_list, default=json_util.default, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Query execution error: {str(e)}"

@tool
def search_users(query: str) -> str:
    """
    Use this tool ONLY for questions about users, their enrollments, or their progress in courses.
    The input must be a valid Python command string using methods like find or aggregate on the 'users_collection' variable.
    """
    print(f"  Executing on 'users' collection: {query}")
    try:
        local_vars = {"users_collection": users_collection, "ObjectId": ObjectId}
        result = eval(query, {"__builtins__": {}}, local_vars)
        if isinstance(result, int): return str(result)
        result_list = list(result)
        if not result_list: return "No users found matching the query."
        return json.dumps(result_list, default=json_util.default, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Query execution error: {str(e)}"

tools = [search_courses, search_users]


SYSTEM_MESSAGE = """
You are a highly specialized MongoDB assistant. Your only goal is to answer user questions by generating a complete, runnable Python command string to be executed by a tool.

**--- YOUR PRIMARY DIRECTIVE ---**
The input for your tools (`Action Input`) MUST be a Python string that starts with `users_collection.` or `courses_collection.`.
NEVER output just a JSON object. ALWAYS output the full command.
Correct format: `users_collection.find({{'full_name': 'کیان پارسایی'}})`
Incorrect format: `{{'full_name': 'کیان پارسایی'}}`

**--- AVAILABLE TOOLS and DATA ---**
You have two tools:
1. `search_courses`: Use for questions about courses, lessons, instructors. Queries MUST start with `courses_collection.`.
2. `search_users`: Use for questions about users, enrollments, progress. Queries MUST start with `users_collection.`.

**`courses` collection structure:**
{{
  "title": "string", "instructor_name": "string", "lessons": [ {{ "title": "string", "content": "string" }} ]
}}

**`users` collection structure:**
{{
  "full_name": "string", "enrollments": [ {{ "course_title": "string", "progress": {{ "percent_complete": "number" }} }} ]
}}

**--- EXAMPLES ---**
Question: "مدرس دوره یادگیری ماشین کیست؟"
Action: search_courses
Action Input: `courses_collection.find({{'title': {{'$regex': 'یادگیری ماشین', '$options': 'i'}}}}, {{'_id': 0, 'instructor_name': 1}})`

Question: "کیان پارسایی در چه دوره‌هایی ثبت نام کرده؟"
Action: search_users
Action Input: `users_collection.find({{'full_name': 'کیان پارسایی'}}, {{'_id': 0, 'enrollments.course_title': 1, 'enrollments.progress': 1}})`
---
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MESSAGE),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

def ask_agent(question):
    print(f"user question: {question}")
    response = agent_executor.invoke({"input": question})
    print("Agent response:")
    print(response["output"])

    print("\n\n========================================")

In [100]:
ask_agent("کیان پارسایی در چه دوره‌هایی و با چه درصدی از پیشرفت ثبت‌نام کرده است؟")

user question: کیان پارسایی در چه دوره‌هایی و با چه درصدی از پیشرفت ثبت‌نام کرده است؟


> Entering new AgentExecutor chain...

Invoking: `search_users` with `{'query': "users_collection.find({'full_name': 'کیان پارسایی'}, {'_id': 0, 'enrollments.course_title': 1, 'enrollments.progress': 1})"}`


  Executing on 'users' collection: users_collection.find({'full_name': 'کیان پارسایی'}, {'_id': 0, 'enrollments.course_title': 1, 'enrollments.progress': 1})
[
  {
    "enrollments": [
      {
        "course_title": "مبانی یادگیری ماشین و علم داده با پایتون",
        "progress": {
          "percent_complete": 100,
          "completed_lessons": 3
        }
      },
      {
        "course_title": "ساخت دستیار هوشمند با مدل‌های زبان بزرگ (LLM) و LangChain",
        "progress": {
          "percent_complete": 33,
          "completed_lessons": 1
        }
      }
    ]
  }
]کیان پارسایی در دوره‌های "مبانی یادگیری ماشین و علم داده با پایتون" و "ساخت دستیار هوشمند با مدل‌های زبان بزرگ (LLM) و Lan

In [101]:
ask_agent("محتوای درس سوم دوره یادگیری ماشین چیست؟")

user question: محتوای درس سوم دوره یادگیری ماشین چیست؟


> Entering new AgentExecutor chain...

Invoking: `search_courses` with `{'query': "courses_collection.find({'title': {'$regex': 'یادگیری ماشین', '$options': 'i'}}, {'_id': 0, 'lessons': 1})"}`


  Executing on 'courses' collection: courses_collection.find({'title': {'$regex': 'یادگیری ماشین', '$options': 'i'}}, {'_id': 0, 'lessons': 1})
[
  {
    "lessons": [
      {
        "lesson_id": {
          "$oid": "68456cba74488871729b2841"
        },
        "title": "درس ۱: مقدمه‌ای بر هوش مصنوعی و یادگیری ماشین",
        "content": "تفاوت بین هوش مصنوعی، یادگیری ماشین و یادگیری عمیق چیست؟ در این درس به این سوالات پاسخ داده و با انواع مختلف مسائل یادگیری ماشین آشنا می‌شویم.",
        "duration_minutes": 20
      },
      {
        "lesson_id": {
          "$oid": "68456cba74488871729b2842"
        },
        "title": "درس ۲: یادگیری نظارت‌شده (Supervised Learning)",
        "content": "با دو دسته اصلی مسائل یادگیری نظارت‌شده، یعنی رگر